# 🔹 YOLO Mimarisi (Genel Katmanlar)

## Backbone
- Görüntüden özellikleri çıkarır (feature extraction).  
- Örnek: CSPDarknet, CSPNet, veya YOLOv8’de kullanılan **CSPDarknet/TorchVision temelli CNN**.

## Neck
- Özellikleri birleştirir ve farklı ölçeklerdeki nesneleri algılar.  
- Kullanılan yapılar: **Feature Pyramid Network (FPN)** veya **PAN (Path Aggregation Network)**.

## Head
- Son katman, **bounding box** ve **sınıf tahminlerini** üretir.  
- Her grid hücresine birkaç anchor box atanır ve o hücre için olası nesneler tahmin edilir.


# 🔹 YOLO’nun Çalışma Prensibi

1. Görüntü **SxS** boyutunda bir ızgaraya bölünür.  
2. Her hücre bir nesnenin merkezini içeriyorsa:
   - Bounding box koordinatları (**x, y, w, h**)  
   - Nesneye ait sınıf olasılıkları  
   - Güven skoru  
   hesaplanır.  
3. Bu tahminler birleştirilir → **NMS (Non-Maximum Suppression)** ile çakışan kutular temizlenir.


# 🔹 YOLOv8 Öne Çıkan Özellikleri

- **Tek model:** Detection, Segmentation ve Classification aynı çatı altında.  
- **Anchor-free detection:** Daha hızlı ve hafif modeller oluşturulabiliyor.  
- Küçük modeller (n, s) → hızlı, gömülü cihazlarda kullanılabilir.  
- Büyük modeller (l, x) → daha doğru ama ağır.


---------

## Mesela YOLO bu kordinatları nasıl öğreniyor ya da nerden biliyor ?

# 🔹 1. Grid Mantığı
- YOLO bir görüntüyü **SxS ızgarasına** böler.  
  Örneğin, 640x640’lık bir resim → 20x20 ızgara hücresi.  
- Her hücre, sorumlu olduğu alanın merkezinde nesne olup olmadığını tahmin eder.



# 🔹 2. Bounding Box Tahmini
- Her hücre **B tane bounding box** tahmin eder. Her kutu için:
  - **x, y** → kutu merkezinin hücreye göre koordinatı (0–1 arasında normalize)  
  - **w, h** → kutunun genişlik ve yüksekliği (resme göre normalize)  
  - **confidence score** → bu kutuda nesne var mı, güven seviyesi  

- Bu değerler, CNN’in son katmanında öğrenilen **feature map** üzerinden doğrudan tahmin edilir.  
- Örn: YOLO “yol” sınıfını tanıyorsa, feature map aktivasyonlarından nesnenin yerini ve boyutunu öğrenir.



# 🔹 3. Sınıf Tahmini
- Her kutu ayrıca **sınıf olasılıklarını** tahmin eder (softmax veya sigmoid).  
- Örn: `[tumsek, kaldirim, yol_calismasi] → [0.1, 0.8, 0.05]` → en yüksek olasılık seçilir.



# 🔹 4. Non-Maximum Suppression (NMS)
- Birden fazla kutu aynı nesneyi tahmin ederse → çakışan kutulardan en yüksek güven skoru olan seçilir.  
- Böylece aynı nesne için birden fazla bounding box çizilmez.



# 🔹 5. Özel Sınıflar (Yol, Tümsek, Kaldırım)
- COCO’da yok → senin **özel datasetin** üzerinden model öğrenir.  
- Eğitim sırasında:  
  - YOLO feature map’e bakar → “bu aktivasyonlar yol veya tümsek nesnesine ait”  
  - Sonuç: tahmin ettiği **koordinatlar ve sınıf**.

📌 **Özet:**  
- YOLO herhangi bir nesnenin koordinatlarını **önceden bilmez**, CNN feature map üzerinden öğrenir.  
- Sen dataset ile modeline örnekleri gösterdikçe “yol nerede, tümsek nerede” öğrenilir.


---
## YOLO'nun kod mantığına gelin beraber bakalım.

### 🔹 1. Grid ve Feature Map

* YOLO, resmi önce bir CNN üzerinden geçirir ve feature map çıkarır:

In [1]:
import torch
import torch.nn as nn

# Basit bir feature extractor
class SimpleBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 16, 3, padding=1)  # RGB -> 16 feature map
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = torch.relu(self.conv(x))
        x = self.pool(x)
        return x

# Örnek giriş
x = torch.randn(1, 3, 64, 64)  # 1 resim, 3 kanal, 64x64
backbone = SimpleBackbone()
feature_map = backbone(x)
print("Feature map boyutu:", feature_map.shape)


Feature map boyutu: torch.Size([1, 16, 32, 32])


 * Çıktı shape: [batch, channels, height, width]

* Bu height x width, YOLO’nun grid mantığına karşılık gelir.

### 🔹 2. Bounding Box Tahmini

* Her grid hücresi için B adet kutu tahmin edilir.

* Kutular için x, y, w, h ve confidence skorları hesaplanır:

In [2]:
B = 2  # hücre başına 2 tahmin
num_classes = 3  # örn: tumsek, kaldirim, yol_calismasi

# Basit head: bounding box + class tahmini
head = nn.Conv2d(16, B*5 + num_classes, 1)  # 5 -> x,y,w,h,conf
pred = head(feature_map)
print("Pred shape:", pred.shape)


Pred shape: torch.Size([1, 13, 32, 32])


* pred.shape → [batch, B*5+num_classes, H, W]

* Her hücreye karşılık gelen tüm tahminler feature map’ten çıkarılır.

### 🔹 3. Tahminleri Grid’e Yerleştirme

* YOLO grid hücreleri üzerinden normalize edilmiş koordinatları çıkarır:

In [3]:
# Örnek: H=32, W=32
H, W = feature_map.shape[2:]
pred = pred.permute(0,2,3,1)  # [batch, H, W, B*5+num_classes]
# Artık her hücre için B kutu ve class skorları hazır


* x,y → hücre içi offset olarak normalize edilir

* w,h → resme göre normalize

### 🔹 4. NMS ve Sınıf Seçimi

* Aynı nesne birden fazla kutu ile tahmin edilirse Non-Maximum Suppression (NMS) uygulanır:

In [4]:
from torchvision.ops import nms

boxes = torch.tensor([[10,10,50,50],[12,12,48,48]], dtype=torch.float)
scores = torch.tensor([0.9, 0.85])
keep = nms(boxes, scores, iou_threshold=0.5)
print("Seçilen kutular:", keep)


Seçilen kutular: tensor([0])


* Bu sayede en iyi tahmin kutusu seçilir.

```python :
💡 Özet:

Görüntü → CNN → feature map

Feature map → Head → B box + confidence + sınıf

Grid hücrelerine dağıt → normalize edilmiş koordinatlar

NMS → fazlalıkları temizle

-----

# YOLO Modeli Katmanları

###  Backbone

* Görüntüden özellik haritası çıkarır (feature map).

* Bu katmanda kontrast, kenar, doku gibi düşük seviye özellikler öğrenilir.

Örnekler: CSPDarknet, ResNet, Darknet-53

* Kod örneği:

```bash 
import torch.nn as nn

class Backbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        return x


###  Neck

* Özellik haritalarını birleştirir ve farklı ölçeklerdeki nesneleri yakalar.

* Kullanılan yapılar: FPN (Feature Pyramid Network), PAN (Path Aggregation Network)

###  Head

* Bounding box ve sınıf tahmini yapan son katmandır.

* Grid hücreleri üzerinden B adet tahmin kutusu üretir:

```bash
x, y → kutu merkezi

w, h → kutu boyutu

confidence → nesne var mı?

sınıf olasılıkları → softmax/sigmoid


```python 
class Head(nn.Module):
    def __init__(self, in_channels, num_classes, B=2):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, B*5 + num_classes, 1)
    
    def forward(self, x):
        return self.conv(x)


##  YOLO Modelinin Tam Yapısı

* Backbone → Neck → Head şeklinde birleştirilir.

```python 

class YOLO(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.backbone = Backbone()
        # Basit bir neck örneği (identity)
        self.neck = nn.Identity()
        self.head = Head(64, num_classes)
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.neck(x)
        x = self.head(x)
        return x


* Çıktı shape: [batch_size, B*5 + num_classes, H, W]

* Bu çıktı her grid hücresi için bounding box ve sınıf bilgilerini içerir.

### Tahminlerin İşlenmesi

* Feature map → Head → grid hücrelerine dağıtılır

* Koordinatlar normalize edilir

* NMS uygulanır → çakışan kutular temizlenir

* Sınıf olasılığına göre en yüksek tahmin seçilir

## 💡 Özet:

* YOLO modeli bir CNN tabanlı feature extractor + özellikleri birleştiren neck + bounding box ve sınıf tahminleri yapan head üçlüsünden oluşur.

* Kendi datasetinle bu yapıyı eğiterek yol, tümsek, kaldırım gibi özel nesneleri tanıyabilirsin.

----

## Yolo Crop İşlemi


# 🔹 Crop İşlemi (YOLO Dataset)

## 1️⃣ Crop’un Amacı
- **Nesneyi büyütmek ve odaklanmak**: Küçük nesneler (tümsek, çukur, çatlak) resmin küçük bir bölümünde olabilir. Crop ile modelin bu nesneleri daha net görmesini sağlarsın.  
- **Veri artırma**: Farklı crop bölgeleri oluşturarak dataset’i çeşitlendirebilirsin.  
- **Gereksiz alanları azaltmak**: Eğer resmin büyük kısmı sadece yol ve boş alan ise, crop ile sadece önemli bölgeyi kullanabilirsin.  



## 2️⃣ Crop Yöntemleri
### A) Etiket Bazlı Crop
- YOLO etiketi (`.txt`) üzerinden xmin, ymin, xmax, ymax koordinatlarını kullanırsın.  
- Crop işlemi direkt bu koordinatlarla yapılır.  

### B) Sabit Bölge Crop
- Eğer tümsek/çukur her zaman resmin aynı bölgesindeyse, **sabit bir crop bölgesi** belirleyebilirsin.  

### C) Rastgele Crop (Augmentation)
- Küçük rastgele kesitler alıp hem nesneyi hem biraz çevresini dahil edersin.  
- Modelin farklı açılar ve pozisyonlar görmesini sağlar.  



## 3️⃣ Crop + YOLO Etiketleri
- Crop sonrası **etiketleri normalize etmeyi unutma**.  
- Örnek:
  - Orijinal resim: 640x640  
  - Bounding box: xmin=50, ymin=60, xmax=150, ymax=120  
  - Crop: 50:150, 60:120 → crop boyutu 100x60  
  - Yeni bounding box → crop içindeki koordinatlara göre normalize edilir:
    ```
    x_center = 0.5
    y_center = 0.5
    width = 1.0
    height = 1.0
    ```

---
## Mesela bir görselde yoldaki çatlaklar var.Biz bunları neye göre kırpacağız ? 
---

# 🔹 Çatlakları Crop Etme Mantığı (YOLO Dataset)

## 1️⃣ Öncelik: Nesnenin Alanını Belirlemek
- Kırpma işlemi için **nesnenin bulunduğu alan** temel alınır.  
- YOLO dataset’inde bu alanlar **etiket dosyalarında (.txt) xmin, ymin, xmax, ymax** olarak verilir.  
- Çatlaklar genellikle uzun ve ince olduğu için birden fazla kutu ile etiketlenmiş olabilir → **her kutuyu ayrı crop veya birleştirilmiş crop** yapabilirsin.



## 2️⃣ Crop Mantığı
### A) Tek Kutulu Crop
- Çatlak küçük ve tek bir bbox ile kapsanabiliyorsa:
  - `xmin, ymin, xmax, ymax` kullan → crop  
  - Kırpılmış resim sadece çatlağı içerir  

### B) Birden Fazla Kutulu Crop
- Çatlak uzun veya birden fazla bbox ile bölünmüşse:
  - Tüm ilgili bbox’ları kapsayan **tek büyük crop** oluştur  
  - Veya her bbox’ı ayrı crop al, farklı örnekler olarak kullan  

### C) Crop + Padding
- Crop sırasında biraz **çevre bırakmak** (padding) faydalıdır:  
  - Model nesnenin kontekstini görür → yol, çevre dokusu  
  - Özellikle çatlak gibi uzun nesnelerde önemlidir  



## 3️⃣ Normalizasyon ve Etiket Güncelleme
- Crop sonrası bbox koordinatlarını **yeni crop boyutuna göre normalize et**:  


```python 
x_center_new = (x_center - crop_xmin) / crop_width
y_center_new = (y_center - crop_ymin) / crop_height
width_new = width / crop_width
height_new = height / crop_height

- Böylece model doğru koordinatları öğrenir.


## 4️⃣ Özet Strateji
1. Her çatlak için etiket dosyalarını kontrol et.  
2. Crop alanını belirle (tek bbox veya birden fazla bbox’ı kapsayan alan).  
3. Crop sırasında padding bırak → yol ve çevre bilgisi kaybolmasın.  
4. Yeni crop için etiketleri normalize et.  
5. Dataset’i çeşitlendirmek için farklı crop bölgeleri ve augmentation ekle.


----
## Görüntüde yolda bulunan çatlak var.Ben bu çatlağı nasıl bulacağım.Mesela kare şeklinde değil de uzun sola yatık halde ? En optimum etiketleme yçntemi nedir ?

---


# 🔹 Yol Üzerindeki Uzun ve Yatık Çatlakların Etiketlenmesi

## 1️⃣ Sorun
- Standart YOLO bounding box’ları **dikdörtgen**.  
- Çatlaklar uzun ve ince → dikdörtgen kutu çok büyük olabilir veya boş alanı kapsayabilir.  
- Bu durum modelin öğrenmesini zorlaştırır.


## 2️⃣ Optimum Etiketleme Yöntemleri

### A) Rotated Bounding Box (Dönebilir Kutular)
- Bazı YOLO sürümleri **rotated bbox** destekler.  
- Çatlağı **uzun eksenine göre döndürerek** kutula → model daha doğru öğrenir.  

### B) Birden Fazla Küçük Dikdörtgen
- Çatlak çok uzun ise, onu **birden fazla küçük dikdörtgen bbox’a böl**.  
- Örn: 3-4 parçaya böl → her parça ayrı etiketlenir.  
- Avantaj: Model uzun ve ince nesneleri daha iyi öğrenir.

### C) Minimum Dikdörtgen
- Çatlağı çevreleyen **en küçük dikdörtgeni** çiz.  
- Eğer çok yatıksa, kutu biraz **padding** ile genişletilebilir → model hem nesneyi hem biraz çevresini görür.  


## 3️⃣ Crop ve Eğitim Açısından
- Uzun çatlağı tek crop içine al → padding bırak.  
- Veya parçalar halinde crop al → dataset’i çeşitlendirmiş olursun.  
- Normalize ederken her bbox’un **crop içindeki koordinatlarını** doğru hesapla.


## 4️⃣ Özet Strateji
1. Çatlak uzun ve yatık → tek dikdörtgen veya birden fazla küçük dikdörtgen.  
2. Padding bırak → yol ve çevre bilgisini kaybetme.  
3. Rotated bbox destekleyen YOLO sürümü kullan → en doğru sonuç.  
4. Dataset’i çeşitlendirmek için parçalama ve augmentation uygula.


### Şimdi sizlerle bir yolda bulunan tümseklerin analizini yapcağız.Yani yolda bulunan tümsekleri etiketleyeceğiz.Sonrasında ise bu etiketlenmiş verileri YOLO'ya entegre ettirip yol ve tümsek olarak etiketleme yapacağız.Sonrasında ise buradan çıkan değerleri CNN modele koyup daha detaylı analizini yapacağız.

#### Bu uygulamaya ve çözüme ulaşmak için lütfen şu dosya yoluna gidiniz.

```python 

* YOLO\YOLO\Uygulamalar\Uygulama - 1 (Yol - Tümsek)\yol_tümsek.ipynb